# Heisenberg 6 Sites &mdash; Analytical Forms

## Import packages

In [1]:
import time, os

import numpy as np
import sympy as sp
from scipy import linalg, sparse

from openfermion.ops import FermionOperator as FO, QubitOperator as QO
from openfermion.linalg import get_sparse_operator, get_ground_state
from openfermion.utils import anticommutator, commutator, save_operator, count_qubits, is_hermitian, hermitian_conjugated

from IPython.display import display, Math, Markdown, Latex, HTML

import matplotlib as mpl
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({
    #"text.usetex": True,
    #"font.family": "serif",
    #"font.serif": ["Computer Modern Roman", "Times", "Palatino"],
    #"font.size": 16,
})
#mpl.rcParams['font.family'] = ['serif', 'sans-serif']
##mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath} \usepackage[T1]{fontenc}'
#mpl.rcParams['text.latex.unicode'] = True
from mpl_toolkits.axes_grid1 import Grid

from qiskit.quantum_info import Statevector

from qutip import Qobj, about, basis, simdiag

In [2]:
diagram_6site = r'''
6-site chain
site index (qubit index)
  6(5)______5(4)
     /      \
1(0)/        \4(3) 
    \        /
     \______?
    2(1)   3(2)
'''
print(diagram_6site)


6-site chain
site index (qubit index)
  6(5)______5(4)
     /      \
1(0)/        \4(3) 
    \        /
     \______?
    2(1)   3(2)



## Define Hamiltonian and SWAP operators

In [3]:
# Heisenberg 6-site Hamiltonian terms in OpenFermion qubit operator form
Lsite = 6
Ham = {}
edges = [(1,2), (2,3), (3,4), (4,5), (5,6), (6,1)]  #site index 1--6, but OpenFermion index 0--5
for ii,jj in edges:
    Ham[(ii,jj)] = QO(f'X{ii-1} X{jj-1}') + QO(f'Y{ii-1} Y{jj-1}') + QO(f'Z{ii-1} Z{jj-1}')

# Full Hamiltonian in qubit operator
HQ = QO()  #initialize with zero operator
for k,v in Ham.items():
    HQ += v
# Convert to sparse numerical matrix, then to numpy array (.A), then to real part (XX, YY, ZZ are purely real)
HM = get_sparse_operator(HQ, n_qubits=Lsite).A.real
eigE_H6, eigV_H6 = np.linalg.eigh(HM)
print('6-site Heisenberg Hamiltonian eigenenergy:',eigE_H6.round(4), sep='\n')
#print([sp.nsimplify(E.round(16)) for E in eigE_H6], sep='\n')

6-site Heisenberg Hamiltonian eigenenergy:
[-11.2111  -8.4721  -8.4721  -8.4721  -6.      -5.1231  -5.1231  -5.1231
  -5.1231  -5.1231  -5.1231  -4.      -4.      -4.      -4.      -4.
  -4.      -2.      -2.      -2.      -2.      -2.      -2.      -2.
  -0.      -0.      -0.      -0.      -0.       0.       0.       0.
   0.       0.       0.4721   0.4721   0.4721   2.       2.       2.
   3.1231   3.1231   3.1231   3.1231   3.1231   3.1231   3.2111   4.
   4.       4.       4.       4.       4.       4.       4.       4.
   4.       6.       6.       6.       6.       6.       6.       6.    ]


In [4]:
HMM = sp.Matrix(HM).applyfunc(sp.nsimplify)

In [5]:
sp.factor(HMM.charpoly().as_expr())

lambda**10*(lambda - 6)**7*(lambda - 4)**10*(lambda - 2)**3*(lambda + 2)**7*(lambda + 4)**6*(lambda + 6)*(lambda**2 + 2*lambda - 16)**6*(lambda**2 + 8*lambda - 36)*(lambda**2 + 8*lambda - 4)**3

In [6]:
symbs = sp.symbols(' '.join(f'|{np.binary_repr(n,width=6)}⟩' for n in range(2**Lsite)))

In [7]:
# faster to only compute eigenvalues
eigs = HMM.eigenvals()
Eskey = list(eigs.keys())
Es = np.array([sp.N(k) for k in Eskey],dtype=float)
idxs = np.argsort(Es)
print(Es[idxs])
d = 0
for n,ii in enumerate(idxs):
    k = Eskey[ii]
    #print(n)
    sp.pprint({k:f'dim={eigs[k]}'})
    d += eigs[k]
print(d)

[-11.21110255  -8.47213595  -6.          -5.12310563  -4.
  -2.           0.           0.47213595   2.           3.12310563
   3.21110255   4.           6.        ]
{-2⋅√13 - 4: dim=1}
{-2⋅√5 - 4: dim=3}
{-6: dim=1}
{-√17 - 1: dim=6}
{-4: dim=6}
{-2: dim=7}
{0: dim=10}
{-4 + 2⋅√5: dim=3}
{2: dim=3}
{-1 + √17: dim=6}
{-4 + 2⋅√13: dim=1}
{4: dim=10}
{6: dim=7}
64


In [8]:
# slower to compute both eigenvalues and eigenvectors
eigs = HMM.eigenvects()
Es = np.array([sp.N(eigs[ii][0]) for ii in range(len(eigs))],dtype=float)
idxs = np.argsort(Es)
print(Es[idxs])
d = 0
for n,ii in enumerate(idxs):
    sp.pprint({eigs[ii][0]:f'dim={eigs[ii][1]}'})
    if n == 0:  #ground state
        print('Ground State (un-normalized):',end='')
        display(np.dot(symbs,eigs[ii][2][0][:]))
        print('Norm:',end='')
        display(sp.simplify(eigs[ii][2][0].norm()))        
    d += eigs[ii][1]
print(d)

[-11.21110255  -8.47213595  -6.          -5.12310563  -4.
  -2.           0.           0.47213595   2.           3.12310563
   3.21110255   4.           6.        ]
{-2⋅√13 - 4: dim=1}
Ground State (un-normalized):

-|000111⟩ + |001011⟩*(3/2 + sqrt(13)/2) + |001101⟩*(-sqrt(13)/2 - 3/2) + |001110⟩ + |010011⟩*(-sqrt(13)/2 - 3/2) + |010101⟩*(sqrt(13) + 4) + |010110⟩*(-sqrt(13)/2 - 3/2) + |011001⟩*(-sqrt(13)/2 - 3/2) + |011010⟩*(3/2 + sqrt(13)/2) - |011100⟩ + |100011⟩ + |100101⟩*(-sqrt(13)/2 - 3/2) + |100110⟩*(3/2 + sqrt(13)/2) + |101001⟩*(3/2 + sqrt(13)/2) + |101010⟩*(-4 - sqrt(13)) + |101100⟩*(3/2 + sqrt(13)/2) - |110001⟩ + |110010⟩*(3/2 + sqrt(13)/2) + |110100⟩*(-sqrt(13)/2 - 3/2) + |111000⟩

Norm:

sqrt(34*sqrt(13) + 130)

{-2⋅√5 - 4: dim=3}
{-6: dim=1}
{-√17 - 1: dim=6}
{-4: dim=6}
{-2: dim=7}
{0: dim=10}
{-4 + 2⋅√5: dim=3}
{2: dim=3}
{-1 + √17: dim=6}
{-4 + 2⋅√13: dim=1}
{4: dim=10}
{6: dim=7}
64


In [9]:
# SWAP operators s_ij
Swaps = {}
for k,v in Ham.items():
    Swaps[k] = 0.5*(v + QO(()))

#nnn swaps (next-nearest neighbors)
for ii in range(6):
    jj = (ii+2)%6
    print((ii+1,jj+1))
    Swaps[(ii+1,jj+1)] = 0.5*(QO(f'X{ii} X{jj}') + QO(f'Y{ii} Y{jj}') + QO(f'Z{ii} Z{jj}') + QO(()))
#nnnn swaps (next-next-nearest neighbors)
for ii in range(3):
    jj = (ii+3)%6
    print((ii+1,jj+1))
    Swaps[(ii+1,jj+1)] = 0.5*(QO(f'X{ii} X{jj}') + QO(f'Y{ii} Y{jj}') + QO(f'Z{ii} Z{jj}') + QO(()))

(1, 3)
(2, 4)
(3, 5)
(4, 6)
(5, 1)
(6, 2)
(1, 4)
(2, 5)
(3, 6)


## Diagonalization in $(\mathbf{S}^2,S_z)$ symmetry basis

In [11]:
# Define symmetry operators in OpenFermion qubit operator form
S2, Sz, Sx, Sy = QO(), QO(), QO(), QO()  #(S^2, Sz, Sx, Sy) qubit operators
for ii in range(Lsite):
    Sz += QO(f'Z{ii}', 0.5)
    Sx += QO(f'X{ii}', 0.5)
    Sy += QO(f'Y{ii}', 0.5)
S2 = Sx*Sx + Sy*Sy + Sz*Sz
#symmetry operators must commute with the full Hamiltonian


#mirror symmetry along 14
m14 = Swaps[(6,2)]*Swaps[(3,5)]
#mirror symmetry perpendicular to 14
# p14 = Swaps[(1,4)]*Swaps[(2,3)]*Swaps[(5,6)]
#inversion or C2
# C2 = Swaps[(1,4)]*Swaps[(2,5)]*Swaps[(3,6)]

print(commutator(S2, HQ), commutator(Sz, HQ), commutator(S2, Sz),
      commutator(HQ, m14), commutator(Sz, m14), commutator(S2, m14), sep=', ')

# Convert to numpy array
SzM = get_sparse_operator(Sz,n_qubits=Lsite).A.real
S2M = get_sparse_operator(S2,n_qubits=Lsite).A.real
m14M = get_sparse_operator(m14,n_qubits=Lsite).A.real
# p14 = get_sparse_operator(p14,n_qubits=Lsite).A.real
# C2 = get_sparse_operator(C2,n_qubits=Lsite).A.real

# Find simultaneous eigenvectors using QuTiP
eigEsym, eigVsym = simdiag([Qobj(S2M), Qobj(SzM), Qobj(HM), Qobj(m14M)])  #<- QuTiP has bugs!!
#^QuTiP bug: with the above ordering for the mutually commuting operators, some computed eigenvalues of m14
#  are not integers (mirror symmetry is involution, so eigenvalues=+/-1). This is similar to the bug here:
#  https://github.com/qutip/qutip/issues/756
#  It's a closed issue but the fix apparently still fails in our case.
#  The follow ordering works (cyclic shift of S2 to the last term).
# eigEsym, eigVsym = simdiag([Qobj(SzM), Qobj(HM), Qobj(m14M), Qobj(S2M)])
#good quantum numbers are all integers here (except for eigen-energies), so round off numerical noise
eigEsym = eigEsym.round(12).T
print(eigEsym.round(3))
#print(np.hstack([np.arange(2**Lsite).reshape((-1,1)), eigEsym]).round(3))

0, 0, 0, 0, 0, 0
[[  0.      0.    -11.211   1.   ]
 [  0.      0.     -6.     -1.   ]
 [ -0.      0.     -2.     -1.   ]
 [  0.      0.     -2.      1.   ]
 [  0.      0.      3.211   1.   ]
 [  2.     -1.     -3.899   0.552]
 [  2.     -1.     -6.869   0.541]
 [  2.     -1.     -5.119  -0.862]
 [  2.     -1.     -4.311   0.448]
 [  2.     -1.     -5.028   0.148]
 [  2.     -1.     -0.315   0.75 ]
 [  2.     -1.      1.427  -0.609]
 [  2.     -1.      2.99   -0.93 ]
 [  2.     -1.      3.123   0.963]
 [  2.     -0.     -8.472   1.   ]
 [  2.      0.     -5.123  -0.874]
 [  2.      0.     -5.123   0.874]
 [  2.     -0.     -4.     -1.   ]
 [  2.     -0.     -4.      1.   ]
 [  2.      0.      0.472   1.   ]
 [  2.     -0.      2.     -1.   ]
 [  2.      0.      3.123  -1.   ]
 [  2.      0.      3.123   1.   ]
 [  2.      1.     -6.787   0.699]
 [  2.      1.     -6.267   0.577]
 [  2.      1.     -5.03   -0.369]
 [  2.      1.     -4.081   0.18 ]
 [  2.      1.     -1.451   0.252]
 [ 

In [10]:
eigEsym = eigEsym[:,[1,3,0,2]]  #re-arrange order: En, S2, Sz, m14
#idx = np.argsort(eigEsym[:,0])
idx = np.lexsort((eigEsym[:,3],eigEsym[:,2],eigEsym[:,1],eigEsym[:,0])) #sort by En, then S2, then Sz, then m14
# idx = np.lexsort((eigEsym[:,3],eigEsym[:,0],eigEsym[:,2],eigEsym[:,1])) #sort (S2,Sz) first
eigEsym = eigEsym[idx,:]
eigVsym = [eigVsym[ii] for ii in idx]
#print(eigEsym.round(3))

#eigenvectors here should be real (imaginary parts are numerical noise); and transpose (.T) to column-vector notation
#eigVsym = np.hstack([vobj.data_as() for vobj in eigVsym]).real #np.array(eigVsym)[:,:,0].T.real ##QuTiP old version
print('   En     s(s+1)  sz     m14')
for ii in range(2**Lsite):
    print('{:6.2f}    {:4.1f}   {:4.1f}   {:4.1f}'.format(*eigEsym[ii,:]))

0, 0, 0, 0, 0, 0
   En     s(s+1)  sz     m14
-11.21     0.0    0.0    1.0
 -8.47     2.0   -1.0    1.0
 -8.47     2.0    0.0    1.0
 -8.47     2.0    1.0    1.0
 -6.00     0.0    0.0   -1.0
 -5.12     2.0   -1.0   -1.0
 -5.12     2.0   -1.0    1.0
 -5.12     2.0    0.0   -1.0
 -5.12     2.0    0.0    1.0
 -5.12     2.0    1.0   -1.0
 -5.12     2.0    1.0    1.0
 -4.00     2.0   -1.0   -1.0
 -4.00     2.0   -1.0    1.0
 -4.00     2.0    0.0   -1.0
 -4.00     2.0    0.0    1.0
 -4.00     2.0    1.0   -1.0
 -4.00     2.0    1.0    1.0
 -2.00     0.0    0.0   -1.0
 -2.00     0.0    0.0    1.0
 -2.00     6.0   -2.0    1.0
 -2.00     6.0   -1.0    1.0
 -2.00     6.0    0.0    1.0
 -2.00     6.0    1.0    1.0
 -2.00     6.0    2.0    1.0
  0.00     6.0   -2.0   -1.0
  0.00     6.0   -2.0    1.0
 -0.00     6.0   -1.0   -1.0
  0.00     6.0   -1.0    1.0
 -0.00     6.0    0.0   -1.0
 -0.00     6.0    0.0    1.0
  0.00     6.0    1.0   -1.0
  0.00     6.0    1.0    1.0
 -0.00     6.0    2.0   -1

In [ ]:
E_eigvals_Luke = \
np.array([-11.21110255,  -8.47213595,  -8.47213595,  -8.47213595,
        -6.        ,  -5.12310563,  -5.12310563,  -5.12310563,
        -5.12310563,  -5.12310563,  -5.12310563,  -4.        ,
        -4.        ,  -4.        ,  -4.        ,  -4.        ,
        -4.        ,  -2.        ,  -2.        ,  -2.        ,
        -2.        ,  -2.        ,  -2.        ,  -2.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.47213595,   0.47213595,
         0.47213595,   2.        ,   2.        ,   2.        ,
         3.12310563,   3.12310563,   3.12310563,   3.12310563,
         3.12310563,   3.12310563,   3.21110255,   4.        ,
         4.        ,   4.        ,   4.        ,   4.        ,
         4.        ,   4.        ,   4.        ,   4.        ,
         4.        ,   6.        ,   6.        ,   6.        ,
         6.        ,   6.        ,   6.        ,   6.])

E_eigvals_Yan = eigEsym[:, 0]
np.allclose(E_eigvals_Luke, E_eigvals_Yan, rtol = 0, atol = 1e-8)

In [ ]:
s_Luke = \
np.array([0.,  2.,  2.,  2.,  0.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  0.,  0.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
          6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  0.,  6.,  6.,  6.,  6.,  6.,
          6.,  6.,  6.,  6.,  6., 12., 12., 12., 12., 12., 12., 12])

s_Yan = eigEsym[:, 1]

np.all(s_Luke == s_Yan)

In [ ]:
sz_eigvals_Yan = eigEsym[:, 2]

sz_eigvals_Luke = \
np.array([ 0., -1.,  0.,  1.,  0., -1., -1.,  0.,  0.,  1.,  1., -1., -1.,
        0.,  0.,  1.,  1.,  0.,  0., -2., -1.,  0.,  1.,  2., -2., -2.,
       -1., -1.,  0.,  0.,  1.,  1.,  2.,  2., -1.,  0.,  1., -1.,  0.,
        1., -1., -1.,  0.,  0.,  1.,  1.,  0., -2., -2., -1., -1.,  0.,
        0.,  1.,  1.,  2.,  2., -3., -2., -1.,  0.,  1.,  2.,  3.])

np.all(sz_eigvals_Luke == sz_eigvals_Yan)

In [ ]:
σ36_eigvals_Luke = \
np.array([ 1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
       -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
       -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,
       -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,
        1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

σ36_eigvals_Yan = eigEsym[:, 3]

np.all(σ36_eigvals_Luke == σ36_eigvals_Yan)